# Reduction Of Dataset Size

**Content**:
1. [Libraries](#libraries)
2. [Gridworld Creation](#gridworld)
3. [Functions](#funct)
4. [Uniform Dataset MLP Performance](#mlpuniform)
5. [MLP performance on 90% of the dataset](#mlpuniform_90)
6. [MLP performance on 80% of the dataset](#mlpuniform_80)
7. [MLP performance on 70% of the dataset](#mlpuniform_70)
8. [MLP performance on 60% of the dataset](#mlpuniform_60)
9. [MLP performance on 50% of the dataset](#mlpuniform_50)
10. [MLP performance on 40% of the dataset](#mlpuniform_40)
11. [MLP performance on 30% of the dataset](#mlpuniform_30)
12. [MLP performance on 20% of the dataset](#mlpuniform_20)

## 1. Libraries <a id = "libraries"> 

In [1]:
import numpy as np
import random
import sys
import pandas as pd
from sklearn.neural_network import MLPRegressor
from sklearn.model_selection import train_test_split, cross_validate, GridSearchCV, validation_curve
from sklearn.model_selection import train_test_split
from sklearn.metrics import make_scorer

## 2. Gridworld Creation <a id = "gridworld">

In [2]:
def randPair(s,e):
    return np.random.randint(s,e), np.random.randint(s,e)

class BoardPiece:

    def __init__(self, name, code, pos):
        self.name = name #name of the piece
        self.code = code #an ASCII character to display on the board
        self.pos = pos #2-tuple e.g. (1,4)

class BoardMask:

    def __init__(self, name, mask, code):
        self.name = name
        self.mask = mask
        self.code = code

    def get_positions(self): #returns tuple of arrays
        return np.nonzero(self.mask)

def zip_positions2d(positions): #positions is tuple of two arrays
    x,y = positions
    return list(zip(x,y))

class GridBoard:

    def __init__(self, size=4):
        self.size = size #Board dimensions, e.g. 4 x 4
        self.components = {} #name : board piece
        self.masks = {}

    def addPiece(self, name, code, pos=(0,0)):
        newPiece = BoardPiece(name, code, pos)
        self.components[name] = newPiece

    #basically a set of boundary elements
    def addMask(self, name, mask, code):
        #mask is a 2D-numpy array with 1s where the boundary elements are
        newMask = BoardMask(name, mask, code)
        self.masks[name] = newMask

    def movePiece(self, name, pos):
        move = True
        for _, mask in self.masks.items():
            if pos in zip_positions2d(mask.get_positions()):
                move = False
        if move:
            self.components[name].pos = pos

    def delPiece(self, name):
        del self.components['name']

    def render(self):
        dtype = '<U2'
        displ_board = np.zeros((self.size, self.size), dtype=dtype)
        displ_board[:] = ' '

        for name, piece in self.components.items():
            displ_board[piece.pos] = piece.code

        for name, mask in self.masks.items():
            displ_board[mask.get_positions()] = mask.code

        return displ_board

    def render_np(self):
        num_pieces = len(self.components) + len(self.masks)
        displ_board = np.zeros((num_pieces, self.size, self.size), dtype=np.uint8)
        layer = 0
        for name, piece in self.components.items():
            pos = (layer,) + piece.pos
            displ_board[pos] = 1
            layer += 1

        for name, mask in self.masks.items():
            x,y = self.masks['boundary'].get_positions()
            z = np.repeat(layer,len(x))
            a = (z,x,y)
            displ_board[a] = 1
            layer += 1
        return displ_board

def addTuple(a,b):
    return tuple([sum(x) for x in zip(a,b)])

In [3]:
class Gridworld:

    def __init__(self, size=4, mode='static'):
        if size >= 4:
            self.board = GridBoard(size=size)
        else:
            print("Minimum board size is 4. Initialized to size 4.")
            self.board = GridBoard(size=4)

        #Add pieces, positions will be updated later
        self.board.addPiece('Player','P',(0,0))
        self.board.addPiece('Goal','+',(0,3))
        self.board.addPiece('Pit','-',(0,3))
        self.board.addPiece('Wall','W',(2,3))

        if mode == 'static':
            self.initGridStatic()
        elif mode == 'player':
            self.initGridPlayer()
        else:
            self.initGridRand()

    #Initialize stationary grid, all items are placed deterministically
    def initGridStatic(self):
        #Setup static pieces
        self.board.components['Player'].pos = randPair(0,self.board.size) #Row, Column
        self.board.components['Goal'].pos = (3,0)
        self.board.components['Pit'].pos = (3,1)
        self.board.components['Wall'].pos = (3,2)
        
        if (not self.validateBoard()):
            #print('Invalid grid. Rebuilding..')
            self.initGridStatic()


    #Check if board is initialized appropriately (no overlapping pieces)
    #also remove impossible-to-win boards
    def validateBoard(self):
        valid = True

        player = self.board.components['Player']
        goal = self.board.components['Goal']
        wall = self.board.components['Wall']
        pit = self.board.components['Pit']

        all_positions = [piece for name,piece in self.board.components.items()]
        all_positions = [player.pos, goal.pos, wall.pos, pit.pos]
        if len(all_positions) > len(set(all_positions)):
            return False

        corners = [(0,0),(0,self.board.size), (self.board.size,0), (self.board.size,self.board.size)]
        #if player is in corner, can it move? if goal is in corner, is it blocked?
        if player.pos in corners or goal.pos in corners:
            val_move_pl = [self.validateMove('Player', addpos) for addpos in [(0,1),(1,0),(-1,0),(0,-1)]]
            val_move_go = [self.validateMove('Goal', addpos) for addpos in [(0,1),(1,0),(-1,0),(0,-1)]]
            if 0 not in val_move_pl or 0 not in val_move_go:
                #print(self.display())
                #print("Invalid board. Re-initializing...")
                valid = False

        return valid

    #Initialize player in random location, but keep wall, goal and pit stationary
    def initGridPlayer(self):
        #height x width x depth (number of pieces)
        self.initGridStatic()
        #place player
        self.board.components['Player'].pos = randPair(0,self.board.size)

        if (not self.validateBoard()):
            #print('Invalid grid. Rebuilding..')
            self.initGridPlayer()

    #Initialize grid so that goal, pit, wall, player are all randomly placed
    def initGridRand(self):
        #height x width x depth (number of pieces)
        self.board.components['Player'].pos = randPair(0,self.board.size)
        self.board.components['Goal'].pos = randPair(0,self.board.size)
        self.board.components['Pit'].pos = randPair(0,self.board.size)
        self.board.components['Wall'].pos = randPair(0,self.board.size)

        if (not self.validateBoard()):
            #print('Invalid grid. Rebuilding..')
            self.initGridRand()

    def validateMove(self, piece, addpos=(0,0)):
        outcome = 0 #0 is valid, 1 invalid, 2 lost game
        pit = self.board.components['Pit'].pos
        wall = self.board.components['Wall'].pos
        new_pos = addTuple(self.board.components[piece].pos, addpos)
        if new_pos == wall:
            outcome = 1 #block move, player can't move to wall
        elif max(new_pos) > (self.board.size-1):    #if outside bounds of board
            outcome = 1
        elif min(new_pos) < 0: #if outside bounds
            outcome = 1
        elif new_pos == pit:
            outcome = 2

        return outcome

    def makeMove(self, action):
        #need to determine what object (if any) is in the new grid spot the player is moving to
        #actions in {u,d,l,r}
        def checkMove(addpos):
            if self.validateMove('Player', addpos) in [0,2]:
                new_pos = addTuple(self.board.components['Player'].pos, addpos)
                self.board.movePiece('Player', new_pos)

        if action == 'u': #up
            checkMove((-1,0))
        elif action == 'd': #down
            checkMove((1,0))
        elif action == 'l': #left
            checkMove((0,-1))
        elif action == 'r': #right
            checkMove((0,1))
        else:
            pass

    def reward(self):
        if (self.board.components['Player'].pos == self.board.components['Pit'].pos):
            return -10
        elif (self.board.components['Player'].pos == self.board.components['Goal'].pos):
            return 10
        else:
            return -1

    def display(self):
        return self.board.render()

In [4]:
action_set = {
    0: 'u',
    1: 'd',
    2: 'l',
    3: 'r',
}

## 3. Functions <a id = 'funct'> 

In [5]:
"""
from2dto1d:
    Args: 
        pos(tuple):2d position of the objects(Player, Wall, Goal, Pit) in the gridworld
    Return:
        return(integer): 1d position of the objects(Player, Wall, Goal, Pit)in the gridworld
""" 
def from2dto1d(pos):
    if pos == '(0, 0)':
        return 0
    if pos == '(0, 1)':
        return 1
    if pos == '(0, 2)':
        return 2
    if pos == '(0, 3)':
        return 3
    if pos == '(1, 0)':
        return 4
    if pos == '(1, 1)':
        return 5
    if pos == '(1, 2)':
        return 6
    if pos == '(1, 3)':
        return 7
    if pos == '(2, 0)':
        return 8
    if pos == '(2, 1)':
        return 9
    if pos == '(2, 2)':
        return 10
    if pos == '(2, 3)':
        return 11
    if pos == '(3, 0)':
        return 12
    if pos == '(3, 1)':
        return 13
    if pos == '(3, 2)':
        return 14
    if pos == '(3, 3)':
        return 15

In [6]:
def from1dto2d(pos):
    if pos == 0:
        return (0, 0)
    if pos == 1:
        return (0, 1)
    if pos == 2:
        return (0, 2)
    if pos == 3:
        return (0, 3)
    if pos == 4:
        return (1, 0)
    if pos == 5:
        return (1, 1)
    if pos == 6:
        return (1, 2)
    if pos == 7:
        return (1, 3)
    if pos == 8:
        return (2, 0)
    if pos == 9:
        return (2, 1)
    if pos == 10:
        return (2, 2)
    if pos == 11:
        return (2, 3)
    if pos == 12:
        return (3, 0)
    if pos == 13:
        return (3, 1)
    if pos == 14:
        return (3, 2)
    if pos == 15:
        return (3, 3)

In [7]:
"""
from_num_to_one_hot_encode:
    Args: 
        num(int): number from 0 to 15 representing the state of the objects(Player, Wall, Goal, Pit) in the gridworld
    Return:
        return(tuple): encoded binary code with the size of 16-bit 
"""
def from_num_to_one_hot_encode(num):
    en0,en1,en2,en3,en4,en5,en6,en7,en8,en9,en10,en11,en12,en13,en14,en15 = 0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
    if num == 0:
        en0 = 1
    if num == 1:
        en1 = 1
    if num == 2:
        en2 = 1
    if num == 3:
        en3 = 1
    if num == 4:
        en4 = 1
    if num == 5:
        en5 = 1
    if num == 6:
        en6 = 1
    if num == 7:
        en7 = 1
    if num == 8:
        en8 = 1
    if num == 9:
        en9 = 1
    if num == 10:
        en10 = 1
    if num == 11:
        en11 = 1
    if num == 12:
        en12 = 1
    if num == 13:
        en13 = 1
    if num == 14:
        en14 = 1
    if num == 15:
        en15 = 1
    return en0,en1,en2,en3,en4,en5,en6,en7,en8,en9,en10,en11,en12,en13,en14,en15

In [8]:
def calculate_smape(actual, predicted) -> float:
    if not all([isinstance(actual, np.ndarray), 
                isinstance(predicted, np.ndarray)]):
        actual, predicted = np.array(actual),
        np.array(predicted)
  
    return round(
        np.mean(
            np.abs(predicted - actual) / 
            ((np.abs(predicted) + np.abs(actual))/2)
        )*100, 2
    )

In [9]:
score = make_scorer(calculate_smape)

In [10]:
def gridworld_test_model(model_b, mode='static', display=True):
    i = 0
    test_game = Gridworld(mode=mode)
    if display:
        print("Initial State:")
        print(test_game.display())
    status = 1
    j = 0
    while(status == 1): #A
        p_curr = from2dto1d((str(test_game.board.components['Player'].pos)))
        pi = from2dto1d((str(test_game.board.components['Pit'].pos)))
        g = from2dto1d((str(test_game.board.components['Goal'].pos)))
        w = from2dto1d((str(test_game.board.components['Wall'].pos)))

        q_value_next = []

        test_game.makeMove('u')
        p = from2dto1d((str(test_game.board.components['Player'].pos)))
        
        if p_curr != p:
            en0,en1,en2,en3,en4,en5,en6,en7,en8,en9,en10,en11,en12,en13,en14,en15 = from_num_to_one_hot_encode(p)
            reward = test_game.reward()
            q_value_next.append(model_b.predict([[p,pi,g,w]])[0])
            test_game.makeMove('d')
        else:
            q_value_next.append(-10)
        
        test_game.makeMove('d')
        p = from2dto1d((str(test_game.board.components['Player'].pos)))
        
        if p_curr != p:
            en0,en1,en2,en3,en4,en5,en6,en7,en8,en9,en10,en11,en12,en13,en14,en15 = from_num_to_one_hot_encode(p)
            reward = test_game.reward()
            q_value_next.append(model_b.predict([[p,pi,g,w]])[0])
            test_game.makeMove('u')
        else:
            q_value_next.append(-10)
        
        test_game.makeMove('l')
        p = from2dto1d((str(test_game.board.components['Player'].pos)))
        

        if p_curr != p:
            test_game.makeMove('r')
            en0,en1,en2,en3,en4,en5,en6,en7,en8,en9,en10,en11,en12,en13,en14,en15 = from_num_to_one_hot_encode(p)
            reward = test_game.reward()
            q_value_next.append(model_b.predict([[p,pi,g,w]])[0])
        else:
            q_value_next.append(-10)
        
        test_game.makeMove('r')
        p = from2dto1d((str(test_game.board.components['Player'].pos)))
        

        if p_curr != p:
            en0,en1,en2,en3,en4,en5,en6,en7,en8,en9,en10,en11,en12,en13,en14,en15 = from_num_to_one_hot_encode(p)
            reward = test_game.reward()
            q_value_next.append(model_b.predict([[p,pi,g,w]])[0])
            test_game.makeMove('l')
        else:
            q_value_next.append(-10)
        
#         print(q_value_next)
        
        
        action_ = np.argmax(q_value_next)
#         print(action_)
        action = action_set[action_]
        
        
    
        if display:
            print('Move #: %s; Taking action: %s' % (i, action))
        test_game.makeMove(action)
        p_next = from2dto1d((str(test_game.board.components['Player'].pos)))

        j += 1
        if display:
            print(test_game.display())
        reward = test_game.reward()
        if reward != -1:
            if reward > 0:
                status = 2
                if display:
                    print("Game won! Reward: %s" % (reward,))
            else:
                status = 0
                if display:
                    print("Game LOST. Reward: %s" % (reward,))
        i += 1
        if (i > 15):
            if display:
                print("Game lost; too many moves.")
            break
    
    win = True if status == 2 else False
    return win

### Dataset

In [18]:
df_uni = pd.read_csv("DatasetAfterUniform_GYM.csv")
df_uni

,Unnamed: 0,Player,Goal,Wall,Pit,Q_Current,Reward_Current,Bellman_Operator
0,0,0,2,1,0,3.076944,-10,2.500878
1,1,2,2,1,0,9.783095,10,10.000000
2,2,3,2,1,0,8.355206,-1,10.000000
3,3,4,2,1,0,3.889865,-1,5.049356
4,4,5,2,1,0,6.721507,-1,5.925810
...,...,...,...,...,...,...,...,...
50395,50395,8,13,14,15,4.916274,-1,6.444922
50396,50396,13,13,14,15,6.726671,10,10.000000
50397,50397,9,13,14,15,8.272136,-1,5.490914
50398,50398,12,13,14,15,4.600275,-1,10.000000


## 4. Unform Dataset MLP Performance <a id = "mlpuniform"> 

### Bellman Opeartor 100%

In [19]:
from sklearn.model_selection import train_test_split
X = df_uni[['Player','Pit', 'Goal', 'Wall']]
y = df_uni[['Bellman_Operator']]

In [20]:
mlp_uniform_bo = MLPRegressor(hidden_layer_sizes=(200,120), max_iter=300, activation = 'relu', solver='adam', random_state=123)

In [ ]:
scores = cross_validate(mlp_uniform_bo, X.values,y.values.ravel(), scoring=score, cv=5, return_train_score=True)
SMAPE_mlp_uniform_bo = np.mean(scores['test_score'])
SMAPE_mlp_uniform_bo

D:\Conda\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:709: UserWarning: Training interrupted by user.
  warnings.warn("Training interrupted by user.")
D:\Conda\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:702: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (300) reached and the optimization hasn't converged yet.
  warnings.warn(
D:\Conda\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:702: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (300) reached and the optimization hasn't converged yet.
  warnings.warn(


In [ ]:
mlp_uniform_bo.fit(X.values,y.values.ravel())

In [ ]:
win_num = 0
for i in range(0,10):
    max_games = 1000
    wins = 0
    for i in range(max_games):
        win = gridworld_test_model(mlp_uniform_bo, 'random', display = False)
        if win:
            wins += 1
    win_perc = float(wins) / float(max_games)
    win_num = win_num + win_perc
    print("Games played: {0}, # of wins: {1}".format(max_games,wins))
    print(win_perc)
    print(win_num)
win_num = win_num / 10
print("Win percentage: {}%".format(win_num*100))

### Q-Value 100%

In [ ]:
from sklearn.model_selection import train_test_split
X = df_uni[['Player','Pit', 'Goal', 'Wall']]
y = df_uni[['Q_Current']]
mlp_uniform_q = MLPRegressor(hidden_layer_sizes=(200,120), max_iter=300, activation = 'relu', solver='adam', random_state=123)
scores = cross_validate(mlp_uniform_q, X.values,y.values.ravel(), scoring=score, cv=5, return_train_score=True)
SMAPE_mlp_uniform_q = np.mean(scores['test_score'])
SMAPE_mlp_uniform_q

In [ ]:
mlp_uniform_q.fit(X.values,y.values.ravel())

In [ ]:
win_num = 0
for i in range(0,10):
    max_games = 1000
    wins = 0
    for i in range(max_games):
        win = gridworld_test_model(mlp_uniform_q, 'random', display = False)
        if win:
            wins += 1
    win_perc = float(wins) / float(max_games)
    win_num = win_num + win_perc
    print("Games played: {0}, # of wins: {1}".format(max_games,wins))
    print(win_perc)
    print(win_num)
win_num = win_num / 10
print("Win percentage: {}%".format(win_num*100))

## 5. MLP performance on 90% of the dataset <a id = "mlpuniform_90"> 

### Bellman Operator 90%

In [ ]:
X = df_uni[['Player','Pit', 'Goal', 'Wall']]
y = df_uni[['Bellman_Operator']]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 1/10, random_state = 0)
mlp_uniform_bo_90 = MLPRegressor(hidden_layer_sizes=(200,120), max_iter=300, activation = 'relu', solver='adam', random_state=123)
scores = cross_validate(mlp_uniform_bo_90, X_train.values,y_train.values.ravel(), scoring=score, cv=5, return_train_score=True)
SMAPE_mlp_uniform_bo_90 = np.mean(scores['test_score'])
SMAPE_mlp_uniform_bo_90

In [ ]:
mlp_uniform_bo_90.fit(X_train.values,y_train.values.ravel())

In [ ]:
win_num = 0
for i in range(0,10):
    max_games = 1000
    wins = 0
    for i in range(max_games):
        win = gridworld_test_model(mlp_uniform_bo_90, 'random', display = False)
        if win:
            wins += 1
    win_perc = float(wins) / float(max_games)
    win_num = win_num + win_perc
    print("Games played: {0}, # of wins: {1}".format(max_games,wins))
    print(win_perc)
    print(win_num)
win_num = win_num / 10
print("Win percentage: {}%".format(win_num*100))

### Q-Value 90%

In [ ]:
X = df_uni[['Player','Pit', 'Goal', 'Wall']]
y = df_uni[['Q_Current']]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 1/10, random_state = 0)
mlp_uniform_q_90 = MLPRegressor(hidden_layer_sizes=(200,120), max_iter=300, activation = 'relu', solver='adam', random_state=123)
scores = cross_validate(mlp_uniform_q_90, X_train.values,y_train.values.ravel(), scoring=score, cv=5, return_train_score=True)
SMAPE_mlp_uniform_q_90 = np.mean(scores['test_score'])
SMAPE_mlp_uniform_q_90

In [ ]:
mlp_uniform_q_90.fit(X_train.values,y_train.values.ravel())

In [ ]:
win_num = 0
for i in range(0,10):
    max_games = 1000
    wins = 0
    for i in range(max_games):
        win = gridworld_test_model(mlp_uniform_q_90, 'random', display = False)
        if win:
            wins += 1
    win_perc = float(wins) / float(max_games)
    win_num = win_num + win_perc
    print("Games played: {0}, # of wins: {1}".format(max_games,wins))
    print(win_perc)
    print(win_num)
win_num = win_num / 10
print("Win percentage: {}%".format(win_num*100))

## 6. MLP performance on 80% of the dataset <a id = "mlpuniform_80"> 

### Bellman Operator 80%

In [ ]:
X = df_uni[['Player','Pit', 'Goal', 'Wall']]
y = df_uni[['Bellman_Operator']]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 2/10, random_state = 0)
mlp_uniform_bo_80 = MLPRegressor(hidden_layer_sizes=(200,120), max_iter=300, activation = 'relu', solver='adam', random_state=123)
scores = cross_validate(mlp_uniform_bo_80, X_train.values,y_train.values.ravel(), scoring=score, cv=5, return_train_score=True)
SMAPE_mlp_uniform_bo_80 = np.mean(scores['test_score'])
SMAPE_mlp_uniform_bo_80

In [ ]:
mlp_uniform_bo_80.fit(X_train.values,y_train.values.ravel())

In [ ]:
win_num = 0
for i in range(0,10):
    max_games = 1000
    wins = 0
    for i in range(max_games):
        win = gridworld_test_model(mlp_uniform_bo_80, 'random', display = False)
        if win:
            wins += 1
    win_perc = float(wins) / float(max_games)
    win_num = win_num + win_perc
    print("Games played: {0}, # of wins: {1}".format(max_games,wins))
    print(win_perc)
    print(win_num)
win_num = win_num / 10
print("Win percentage: {}%".format(win_num*100))

### Q-Value 80%

In [ ]:
X = df_uni[['Player','Pit', 'Goal', 'Wall']]
y = df_uni[['Q_Current']]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 2/10, random_state = 0)
mlp_uniform_q_80 = MLPRegressor(hidden_layer_sizes=(200,120), max_iter=300, activation = 'relu', solver='adam', random_state=123)
scores = cross_validate(mlp_uniform_q_80, X_train.values,y_train.values.ravel(), scoring=score, cv=5, return_train_score=True)
SMAPE_mlp_uniform_q_80 = np.mean(scores['test_score'])
SMAPE_mlp_uniform_q_80

In [ ]:
mlp_uniform_q_80.fit(X_train.values,y_train.values.ravel())

In [ ]:
win_num = 0
for i in range(0,10):
    max_games = 1000
    wins = 0
    for i in range(max_games):
        win = gridworld_test_model(mlp_uniform_q_80, 'random', display = False)
        if win:
            wins += 1
    win_perc = float(wins) / float(max_games)
    win_num = win_num + win_perc
    print("Games played: {0}, # of wins: {1}".format(max_games,wins))
    print(win_perc)
    print(win_num)
win_num = win_num / 10
print("Win percentage: {}%".format(win_num*100))

## 7. MLP performance on 70% of the dataset <a id = "mlpuniform_70"> 

### Bellman Operator 70%

In [ ]:
X = df_uni[['Player','Pit', 'Goal', 'Wall']]
y = df_uni[['Bellman_Operator']]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 3/10, random_state = 0)
mlp_uniform_bo_70 = MLPRegressor(hidden_layer_sizes=(200,120), max_iter=300, activation = 'relu', solver='adam', random_state=123)
scores = cross_validate(mlp_uniform_bo_70, X_train.values,y_train.values.ravel(), scoring=score, cv=5, return_train_score=True)
SMAPE_mlp_uniform_bo_70 = np.mean(scores['test_score'])
SMAPE_mlp_uniform_bo_70

In [ ]:
mlp_uniform_bo_70.fit(X_train.values,y_train.values.ravel())

In [ ]:
win_num = 0
for i in range(0,10):
    max_games = 1000
    wins = 0
    for i in range(max_games):
        win = gridworld_test_model(mlp_uniform_bo_70, 'random', display = False)
        if win:
            wins += 1
    win_perc = float(wins) / float(max_games)
    win_num = win_num + win_perc
    print("Games played: {0}, # of wins: {1}".format(max_games,wins))
    print(win_perc)
    print(win_num)
win_num = win_num / 10
print("Win percentage: {}%".format(win_num*100))

### Q-Value 70%

In [ ]:
X = df_uni[['Player','Pit', 'Goal', 'Wall']]
y = df_uni[['Q_Current']]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 3/10, random_state = 0)
mlp_uniform_q_70 = MLPRegressor(hidden_layer_sizes=(200,120), max_iter=300, activation = 'relu', solver='adam', random_state=123)
scores = cross_validate(mlp_uniform_q_70, X_train.values,y_train.values.ravel(), scoring=score, cv=5, return_train_score=True)
SMAPE_mlp_uniform_q_70 = np.mean(scores['test_score'])
SMAPE_mlp_uniform_q_70

In [ ]:
mlp_uniform_q_70.fit(X_train.values,y_train.values.ravel())

In [ ]:
win_num = 0
for i in range(0,10):
    max_games = 1000
    wins = 0
    for i in range(max_games):
        win = gridworld_test_model(mlp_uniform_q_70, 'random', display = False)
        if win:
            wins += 1
    win_perc = float(wins) / float(max_games)
    win_num = win_num + win_perc
    print("Games played: {0}, # of wins: {1}".format(max_games,wins))
    print(win_perc)
    print(win_num)
win_num = win_num / 10
print("Win percentage: {}%".format(win_num*100))

## 8. MLP performance on 60% of the dataset <a id = "mlpuniform_60"> 

### Bellman Operator 60%

In [ ]:
X = df_uni[['Player','Pit', 'Goal', 'Wall']]
y = df_uni[['Bellman_Operator']]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 4/10, random_state = 0)
mlp_uniform_bo_60 = MLPRegressor(hidden_layer_sizes=(200,120), max_iter=300, activation = 'relu', solver='adam', random_state=123)
scores = cross_validate(mlp_uniform_bo_60, X_train.values,y_train.values.ravel(), scoring=score, cv=5, return_train_score=True)
SMAPE_mlp_uniform_bo_60 = np.mean(scores['test_score'])
SMAPE_mlp_uniform_bo_60

In [ ]:
mlp_uniform_bo_60.fit(X_train.values,y_train.values.ravel())

In [ ]:
win_num = 0
for i in range(0,10):
    max_games = 1000
    wins = 0
    for i in range(max_games):
        win = gridworld_test_model(mlp_uniform_bo_60, 'random', display = False)
        if win:
            wins += 1
    win_perc = float(wins) / float(max_games)
    win_num = win_num + win_perc
    print("Games played: {0}, # of wins: {1}".format(max_games,wins))
    print(win_perc)
    print(win_num)
win_num = win_num / 10
print("Win percentage: {}%".format(win_num*100))

### Q-Value 60%

In [ ]:
X = df_uni[['Player','Pit', 'Goal', 'Wall']]
y = df_uni[['Q_Current']]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 4/10, random_state = 0)
mlp_uniform_q_60 = MLPRegressor(hidden_layer_sizes=(200,120), max_iter=300, activation = 'relu', solver='adam', random_state=123)
scores = cross_validate(mlp_uniform_q_60, X_train.values,y_train.values.ravel(), scoring=score, cv=5, return_train_score=True)
SMAPE_mlp_uniform_q_60 = np.mean(scores['test_score'])
SMAPE_mlp_uniform_q_60

In [ ]:
mlp_uniform_q_60.fit(X_train.values,y_train.values.ravel())

In [ ]:
win_num = 0
for i in range(0,10):
    max_games = 1000
    wins = 0
    for i in range(max_games):
        win = gridworld_test_model(mlp_uniform_q_60, 'random', display = False)
        if win:
            wins += 1
    win_perc = float(wins) / float(max_games)
    win_num = win_num + win_perc
    print("Games played: {0}, # of wins: {1}".format(max_games,wins))
    print(win_perc)
    print(win_num)
win_num = win_num / 10
print("Win percentage: {}%".format(win_num*100))

## 9. MLP performance on 50% of the dataset <a id = "mlpuniform_50"> 

### Bellman Operator 50%

In [ ]:
X = df_uni[['Player','Pit', 'Goal', 'Wall']]
y = df_uni[['Bellman_Operator']]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 5/10, random_state = 0)
mlp_uniform_bo_50 = MLPRegressor(hidden_layer_sizes=(200,120), max_iter=300, activation = 'relu', solver='adam', random_state=123)
scores = cross_validate(mlp_uniform_bo_50, X_train.values,y_train.values.ravel(), scoring=score, cv=5, return_train_score=True)
SMAPE_mlp_uniform_bo_50 = np.mean(scores['test_score'])
SMAPE_mlp_uniform_bo_50

In [ ]:
mlp_uniform_bo_50.fit(X_train.values,y_train.values.ravel())

In [ ]:
win_num = 0
for i in range(0,10):
    max_games = 1000
    wins = 0
    for i in range(max_games):
        win = gridworld_test_model(mlp_uniform_bo_50, 'random', display = False)
        if win:
            wins += 1
    win_perc = float(wins) / float(max_games)
    win_num = win_num + win_perc
    print("Games played: {0}, # of wins: {1}".format(max_games,wins))
    print(win_perc)
    print(win_num)
win_num = win_num / 10
print("Win percentage: {}%".format(win_num*100))

### Q-Value 50%

In [ ]:
X = df_uni[['Player','Pit', 'Goal', 'Wall']]
y = df_uni[['Q_Current']]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 5/10, random_state = 0)
mlp_uniform_q_50 = MLPRegressor(hidden_layer_sizes=(200,120), max_iter=300, activation = 'relu', solver='adam', random_state=123)
scores = cross_validate(mlp_uniform_q_50, X_train.values,y_train.values.ravel(), scoring=score, cv=5, return_train_score=True)
SMAPE_mlp_uniform_q_50 = np.mean(scores['test_score'])
SMAPE_mlp_uniform_q_50

In [ ]:
mlp_uniform_q_50.fit(X_train.values,y_train.values.ravel())

In [ ]:
win_num = 0
for i in range(0,10):
    max_games = 1000
    wins = 0
    for i in range(max_games):
        win = gridworld_test_model(mlp_uniform_q_50, 'random', display = False)
        if win:
            wins += 1
    win_perc = float(wins) / float(max_games)
    win_num = win_num + win_perc
    print("Games played: {0}, # of wins: {1}".format(max_games,wins))
    print(win_perc)
    print(win_num)
win_num = win_num / 10
print("Win percentage: {}%".format(win_num*100))

## 10. MLP performance on 40% of the dataset <a id = "mlpuniform_40"> 

### Bellman Opearator 40%

In [ ]:
X = df_uni[['Player','Pit', 'Goal', 'Wall']]
y = df_uni[['Bellman_Operator']]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 6/10, random_state = 0)
mlp_uniform_bo_40 = MLPRegressor(hidden_layer_sizes=(200,120), max_iter=300, activation = 'relu', solver='adam', random_state=123)
scores = cross_validate(mlp_uniform_bo_40, X_train.values,y_train.values.ravel(), scoring=score, cv=5, return_train_score=True)
SMAPE_mlp_uniform_bo_40 = np.mean(scores['test_score'])
SMAPE_mlp_uniform_bo_40

In [ ]:
mlp_uniform_bo_40.fit(X_train.values,y_train.values.ravel())

In [ ]:
win_num = 0
for i in range(0,10):
    max_games = 1000
    wins = 0
    for i in range(max_games):
        win = gridworld_test_model(mlp_uniform_bo_40, 'random', display = False)
        if win:
            wins += 1
    win_perc = float(wins) / float(max_games)
    win_num = win_num + win_perc
    print("Games played: {0}, # of wins: {1}".format(max_games,wins))
    print(win_perc)
    print(win_num)
win_num = win_num / 10
print("Win percentage: {}%".format(win_num*100))

### Q-Value 40%

In [ ]:
X = df_uni[['Player','Pit', 'Goal', 'Wall']]
y = df_uni[['Q_Current']]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 6/10, random_state = 0)
mlp_uniform_q_40 = MLPRegressor(hidden_layer_sizes=(200,120), max_iter=300, activation = 'relu', solver='adam', random_state=123)
scores = cross_validate(mlp_uniform_q_40, X_train.values,y_train.values.ravel(), scoring=score, cv=5, return_train_score=True)
SMAPE_mlp_uniform_q_40 = np.mean(scores['test_score'])
SMAPE_mlp_uniform_q_40

In [ ]:
mlp_uniform_q_40.fit(X_train.values,y_train.values.ravel())

In [ ]:
win_num = 0
for i in range(0,10):
    max_games = 1000
    wins = 0
    for i in range(max_games):
        win = gridworld_test_model(mlp_uniform_q_40, 'random', display = False)
        if win:
            wins += 1
    win_perc = float(wins) / float(max_games)
    win_num = win_num + win_perc
    print("Games played: {0}, # of wins: {1}".format(max_games,wins))
    print(win_perc)
    print(win_num)
win_num = win_num / 10
print("Win percentage: {}%".format(win_num*100))

## 11. MLP performance on 30% of the dataset <a id = "mlpuniform_30"> 

### Bellman Operator 30%

In [ ]:
X = df_uni[['Player','Pit', 'Goal', 'Wall']]
y = df_uni[['Bellman_Operator']]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 7/10, random_state = 0)
mlp_uniform_bo_30 = MLPRegressor(hidden_layer_sizes=(200,120), max_iter=300, activation = 'relu', solver='adam', random_state=123)
scores = cross_validate(mlp_uniform_bo_30, X_train.values,y_train.values.ravel(), scoring=score, cv=5, return_train_score=True)
SMAPE_mlp_uniform_bo_30 = np.mean(scores['test_score'])
SMAPE_mlp_uniform_bo_30

In [ ]:
mlp_uniform_bo_30.fit(X_train.values,y_train.values.ravel())

In [ ]:
win_num = 0
for i in range(0,10):
    max_games = 1000
    wins = 0
    for i in range(max_games):
        win = gridworld_test_model(mlp_uniform_bo_30, 'random', display = False)
        if win:
            wins += 1
    win_perc = float(wins) / float(max_games)
    win_num = win_num + win_perc
    print("Games played: {0}, # of wins: {1}".format(max_games,wins))
    print(win_perc)
    print(win_num)
win_num = win_num / 10
print("Win percentage: {}%".format(win_num*100))

### Q-Value 30%

In [ ]:
X = df_uni[['Player','Pit', 'Goal', 'Wall']]
y = df_uni[['Q_Current']]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 7/10, random_state = 0)
mlp_uniform_q_30 = MLPRegressor(hidden_layer_sizes=(200,120), max_iter=300, activation = 'relu', solver='adam', random_state=123)
scores = cross_validate(mlp_uniform_q_30, X_train.values,y_train.values.ravel(), scoring=score, cv=5, return_train_score=True)
SMAPE_mlp_uniform_q_30 = np.mean(scores['test_score'])
SMAPE_mlp_uniform_q_30

In [ ]:
mlp_uniform_q_30.fit(X_train.values,y_train.values.ravel())

In [ ]:
win_num = 0
for i in range(0,10):
    max_games = 1000
    wins = 0
    for i in range(max_games):
        win = gridworld_test_model(mlp_uniform_q_30, 'random', display = False)
        if win:
            wins += 1
    win_perc = float(wins) / float(max_games)
    win_num = win_num + win_perc
    print("Games played: {0}, # of wins: {1}".format(max_games,wins))
    print(win_perc)
    print(win_num)
win_num = win_num / 10
print("Win percentage: {}%".format(win_num*100))

## 12. MLP performance on 20% of the dataset <a id = "mlpuniform_20"> 

### Bellman Operator 20%

In [ ]:
X = df_uni[['Player','Pit', 'Goal', 'Wall']]
y = df_uni[['Bellman_Operator']]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 8/10, random_state = 0)
mlp_uniform_bo_20 = MLPRegressor(hidden_layer_sizes=(200,120), max_iter=300, activation = 'relu', solver='adam', random_state=123)
scores = cross_validate(mlp_uniform_bo_20, X_train.values,y_train.values.ravel(), scoring=score, cv=5, return_train_score=True)
SMAPE_mlp_uniform_bo_20 = np.mean(scores['test_score'])
SMAPE_mlp_uniform_bo_20

In [ ]:
mlp_uniform_bo_20.fit(X_train.values,y_train.values.ravel())

In [ ]:
win_num = 0
for i in range(0,10):
    max_games = 1000
    wins = 0
    for i in range(max_games):
        win = gridworld_test_model(mlp_uniform_bo_20, 'random', display = False)
        if win:
            wins += 1
    win_perc = float(wins) / float(max_games)
    win_num = win_num + win_perc
    print("Games played: {0}, # of wins: {1}".format(max_games,wins))
    print(win_perc)
    print(win_num)
win_num = win_num / 10
print("Win percentage: {}%".format(win_num*100))

### Q-Value 20%

In [ ]:
X = df_uni[['Player','Pit', 'Goal', 'Wall']]
y = df_uni[['Q_Current']]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 8/10, random_state = 0)
mlp_uniform_q_20 = MLPRegressor(hidden_layer_sizes=(200,120), max_iter=300, activation = 'relu', solver='adam', random_state=123)
scores = cross_validate(mlp_uniform_q_20, X_train.values,y_train.values.ravel(), scoring=score, cv=5, return_train_score=True)
SMAPE_mlp_uniform_q_20 = np.mean(scores['test_score'])
SMAPE_mlp_uniform_q_20

In [ ]:
mlp_uniform_q_20.fit(X_train.values,y_train.values.ravel())

In [ ]:
win_num = 0
for i in range(0,10):
    max_games = 1000
    wins = 0
    for i in range(max_games):
        win = gridworld_test_model(mlp_uniform_q_20, 'random', display = False)
        if win:
            wins += 1
    win_perc = float(wins) / float(max_games)
    win_num = win_num + win_perc
    print("Games played: {0}, # of wins: {1}".format(max_games,wins))
    print(win_perc)
    print(win_num)
win_num = win_num / 10
print("Win percentage: {}%".format(win_num*100))

In [ ]:
X_train